<a href="https://colab.research.google.com/github/sappyb/Codes-Doc/blob/master/Model_flow_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import math

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
images_dir = '/content/gdrive/My Drive/rate_distribution'
model_dir = '/content/gdrive/My Drive/rate_distribution/mymodel_1'
model_dir = '/content/gdrive/My Drive/rate_distribution/mymodel_2'
work_dir = '/content/gdrive/My Drive/rate_distribution'

In [ ]:
from collections import Counter
from sklearn.model_selection import train_test_split


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#applications = ['amg', 'kripke', 'laghos', 'stencil4d', 'subcomm3d', 'sw4lite']
#applications = ['amg', 'laghos', 'stencil4d', 'kripke']
applications = ['subcomm3d']
ranks = ['32', '64', '128', '256', '512']

In [ ]:
prepivot_train = pd.DataFrame()
prepivot_test = pd.DataFrame()
number_of_flows_per_application = []
for i in applications:
  for j in ranks:
    f_name = '/content/gdrive/My Drive/csvs/full-{}-{}.csv'.format(i, j)

    data_a = pd.read_csv(f_name)
    data_a = data_a.sample(frac=1).reset_index(drop=True)
    if j == '32':
      if i == 'amg':
        data_a = data_a.head(int(100 * 3.5))
      elif i == 'kripke':
        data_a = data_a.head(int(100 * 4))
        data_a = pd.DataFrame(pd.np.repeat(data_a.values,2,axis=0),columns=data_a.columns)
      elif i == 'laghos':
        data_a = data_a.head(int(100 * 6))
      else:
        data_a = data_a.head(int(100))
    if j == '64':
      if i == 'amg':
        data_a = data_a.head(int(220 * 3.5))
      elif i == 'kripke':
        data_a = data_a.head(int(220 * 4))
        data_a = pd.DataFrame(pd.np.repeat(data_a.values,2,axis=0),columns=data_a.columns)
      elif i == 'laghos':
        data_a = data_a.head(int(220 * 6))
      else:
        data_a = data_a.head(int(220))
    if j == '128':
      if i == 'amg':
        data_a = data_a.head(int(460 * 3.5))
      if i == 'kripke':
        data_a = data_a.head(int(460 * 4))
        data_a = pd.DataFrame(pd.np.repeat(data_a.values,2,axis=0),columns=data_a.columns)
      if i == 'laghos':
        data_a = data_a.head(int(460 * 6))
      else:
        data_a = data_a.head(int(460))
    if j == '256':
      if i == 'amg':
        data_a = data_a.head(int(950 * 3.5))
      elif i == 'kripke':
        data_a = data_a.head(int(950 * 4))
        data_a = pd.DataFrame(pd.np.repeat(data_a.values,2,axis=0),columns=data_a.columns)
      elif i == 'laghos':
        data_a = data_a.head(int(950 * 6))
      else:
        data_a = data_a.head(int(950))
    if j == '512':
      if i == 'amg':
        data_a = data_a.head(int(1950 * 3.5))
      elif i == 'kripke':
        data_a = data_a.head(int(1950 * 4))
        data_a = pd.DataFrame(pd.np.repeat(data_a.values,2,axis=0),columns=data_a.columns)
      elif i == 'laghos':
        data_a = data_a.head(int(1950 * 6))
      else:
        data_a = data_a.head(int(1950))
    data_list = []
    for k in range(300):
      data_list.append('Data_{}'.format(k))
    data_a['result'] = data_a[data_list].sum(axis=1)
    elephant = []
    for z in data_a['result'].to_list():
        if z > (1 * 10**8):
          elephant.append(1)
        else:
          elephant.append(0)
    data_a['Is_Elephant'] = elephant
    data_a['Application'] = i
    data_a['Rank'] = j
    prepivot_train = pd.concat([prepivot_train, data_a])
    number_of_flows_per_application.append(data_a.shape[0])
    elephant.clear()
print(prepivot_train)


      Unnamed: 0  Flow_id        Time_0    Rate_0  Packet_Rate_0      Data_0  \
0              0       53  5.499422e+08  0.000000   1.000000e-08         0.0   
1              0       83  5.499422e+08  0.125831   3.084000e-05  12583104.0   
2              0      140  5.499422e+08  0.125831   3.084000e-05  12583104.0   
3              0      139  5.499422e+08  0.136317   3.341000e-05  13631696.0   
4              0       60  5.499422e+08  0.000000   1.000000e-08         0.0   
...          ...      ...           ...       ...            ...         ...   
1945           0     2120  1.070663e+09  0.115345   2.827000e-05  11534512.0   
1946           0     5018  1.070663e+09  0.115345   2.827000e-05  11534512.0   
1947           0      305  1.070663e+09  0.000000   1.000000e-08         0.0   
1948           0     1516  1.070663e+09  0.115345   2.827000e-05  11534512.0   
1949           0     6947  1.070663e+09  0.115345   2.827000e-05  11534512.0   

            Time_1    Rate_1  Packet_Ra

In [ ]:
prepivot_train = prepivot_train.fillna(0)

In [ ]:
num_of_samples_train = prepivot_train.shape[0]
num_of_time_steps = 301
num_of_features = 1

In [ ]:
labels_train = prepivot_train['Is_Elephant'].to_list()

In [ ]:
for i in range(301):
  prepivot_train = prepivot_train.drop('Time_{}'.format(i), axis='columns')
  prepivot_train = prepivot_train.drop('Rate_{}'.format(i), axis='columns')
  prepivot_train = prepivot_train.drop('Packet_Rate_{}'.format(i), axis='columns')

In [ ]:
prepivot_train.iloc[:, 2:-4]

,Data_0,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Data_8,Data_9,...,Data_291,Data_292,Data_293,Data_294,Data_295,Data_296,Data_297,Data_298,Data_299,Data_300
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12583104.0,2097184.0,3145776.0,3145776.0,2097184.0,2097184.0,3145776.0,3145776.0,1048592.0,2097184.0,...,2097184.0,1048592.0,2097184.0,3145776.0,2097184.0,2097184.0,2097184.0,3145776.0,3145776.0,3145776.0
2,12583104.0,2097184.0,3145776.0,3145776.0,2097184.0,2097184.0,3145776.0,3145776.0,1048592.0,2097184.0,...,2097184.0,1048592.0,2097200.0,3145760.0,2097184.0,2097184.0,2097184.0,3145776.0,3145776.0,3145776.0
3,13631696.0,2097200.0,3145760.0,2097184.0,2097184.0,3145776.0,3145776.0,2097184.0,2097184.0,2097184.0,...,3145776.0,2097184.0,2097184.0,2097184.0,2818096.0,2424864.0,2097184.0,2097184.0,3145776.0,2097184.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,11534512.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,...,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,2097184.0
1946,11534512.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,2097184.0,2097184.0,0.0,1048592.0,...,1048592.0,1048592.0,1048608.0,1048592.0,2097168.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0
1947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1948,11534512.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,2097184.0,1048592.0,1048592.0,...,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,2097184.0,1048592.0,1048592.0,1048592.0,1048592.0


In [ ]:
prepivot_train.iloc[:, :]

,Unnamed: 0,Flow_id,Data_0,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,...,Data_295,Data_296,Data_297,Data_298,Data_299,Data_300,result,Is_Elephant,Application,Rank
0,0,53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,subcomm3d,32
1,0,83,12583104.0,2097184.0,3145776.0,3145776.0,2097184.0,2097184.0,3145776.0,3145776.0,...,2097184.0,2097184.0,2097184.0,3145776.0,3145776.0,3145776.0,730868624.0,1,subcomm3d,32
2,0,140,12583104.0,2097184.0,3145776.0,3145776.0,2097184.0,2097184.0,3145776.0,3145776.0,...,2097184.0,2097184.0,2097184.0,3145776.0,3145776.0,3145776.0,730868624.0,1,subcomm3d,32
3,0,139,13631696.0,2097200.0,3145760.0,2097184.0,2097184.0,3145776.0,3145776.0,2097184.0,...,2818096.0,2424864.0,2097184.0,2097184.0,3145776.0,2097184.0,730868624.0,1,subcomm3d,32
4,0,60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,subcomm3d,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,0,2120,11534512.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,...,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,2097184.0,343938176.0,1,subcomm3d,512
1946,0,5018,11534512.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,2097184.0,2097184.0,...,2097168.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,344986768.0,1,subcomm3d,512
1947,0,305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,subcomm3d,512
1948,0,1516,11534512.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,1048592.0,2097184.0,...,1048592.0,2097184.0,1048592.0,1048592.0,1048592.0,1048592.0,344986768.0,1,subcomm3d,512


In [ ]:
data_test_probe = np.array(prepivot_train.iloc[:, 2:-4])

In [ ]:
row_with_max_value = np.amax(data_test_probe, axis=0)
row_with_max_value

array([ 8388672., 12583008., 12583008., 12583008., 12583008., 12583008.,
       12583008., 12583008., 14639200., 12583008., 14680160., 13840480.,
       12583008., 12582976., 12582976., 12882016., 14549088., 13746272.,
       12582976., 13693024., 12583008., 15614048., 13000800., 12583008.,
       12583008., 12583008., 12763232., 12583008., 14127200., 15515744.,
       13979744., 12583008., 12583008., 13750368., 12583008., 12808288.,
       12583008., 14159968., 16777312., 12583008., 13520992., 12583008.,
       13594720., 12951648., 13045856., 12583008., 14647392., 12583008.,
       12583008., 12583008., 13676640., 15302752., 12583008., 14057568.,
       14184544., 13709408., 13652064., 13258848., 13467744., 12583008.,
       12582976., 13926496., 14168160., 13127776., 13185120., 12808288.,
       12583008., 12902496., 12582976., 12808288., 12583008., 13594720.,
       12583008., 13262944., 12583008., 13201504., 14319712., 12583008.,
       12869728., 13074528., 12583008., 12582976., 

In [ ]:
np.sum(row_with_max_value) // 300

13331194.0

In [ ]:
np.amax(data_test_probe)

16777312.0

In [ ]:
np.amax(np.array(prepivot_train['result']))

2773562048.0

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
xp_train = scaler.fit_transform(prepivot_train.iloc[:, 2:-4])
#xp_train = prepivot_train.iloc[:, 2:-4]

Arranging the data from 2D to 3D,

X axis = Sample
Y axis = Time_step
Z axis = Features


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing import sequence
from keras.regularizers import l2
from keras import optimizers
import tensorflow as tf

In [ ]:
epochs = 50
batch_size = 1024
time_steps = 301

In [ ]:
labels_1_train = [[i] for i in labels_train]
labels_1_train = np.array(labels_1_train)

In [ ]:
train_X = xp_train
train_Y = labels_1_train

In [ ]:
model = Sequential()
#model.add(Dense(1, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
hist = model.fit(train_X, train_Y, validation_split=0.2, epochs=epochs, batch_size=batch_size)
print(model.summary())
model.save('model', save_format='tf')

Epoch 1/50
32/32 [==============================] - 1s 10ms/step - loss: 0.5840 - accuracy: 0.9869 - val_loss: 0.5699 - val_accuracy: 1.0000
Epoch 2/50
32/32 [==============================] - 0s 4ms/step - loss: 0.4640 - accuracy: 1.0000 - val_loss: 0.4523 - val_accuracy: 1.0000
Epoch 3/50
32/32 [==============================] - 0s 8ms/step - loss: 0.3709 - accuracy: 1.0000 - val_loss: 0.3625 - val_accuracy: 1.0000
Epoch 4/50
32/32 [==============================] - 0s 7ms/step - loss: 0.3005 - accuracy: 0.9985 - val_loss: 0.2948 - val_accuracy: 1.0000
Epoch 5/50
32/32 [==============================] - 0s 14ms/step - loss: 0.2470 - accuracy: 0.9975 - val_loss: 0.2434 - val_accuracy: 1.0000
Epoch 6/50
32/32 [==============================] - 0s 12ms/step - loss: 0.2062 - accuracy: 0.9975 - val_loss: 0.2039 - val_accuracy: 1.0000
Epoch 7/50
32/32 [==============================] - 1s 27ms/step - loss: 0.1744 - accuracy: 0.9975 - val_loss: 0.1731 - val_accuracy: 1.0000
Epoch 8/50
32/32

Testing with other data, incomplete


In [ ]:
import tensorflow as tf

In [ ]:
#model = tf.keras.models.load_model(model_dir)
applications = ['subcomm3d']
ranks = ['32', '64', '128', '256', '512']
app = []
accuracy = []
count = 0
current = 0
print(number_of_flows_per_application)
for i in applications:
  for j in ranks:
    label_test = prepivot_train.iloc[current:current+number_of_flows_per_application[count], :]['Is_Elephant'].to_list()
    dataq_test = xp_train
    labels_1_test = [[i] for i in label_test]
    labels_1_test = np.array(labels_1_test)
    test_X_1 = dataq_test[current:current+number_of_flows_per_application[count], :]
    test_Y_1 = labels_1_train[current:current+number_of_flows_per_application[count], :]
    scores = model.evaluate(test_X_1, test_Y_1, verbose=0)
    predict= model.predict(test_X_1, verbose=1)
    print(predict.shape)
    app.append('{}-{}'.format(i, j))
    accuracy.append(scores[1]*100)
    print("Accuracy of {} {}: {}".format(i, j, scores[1]*100))
    current += number_of_flows_per_application[count]
    count += 1

[100, 220, 460, 950, 1950]
4/4 [==============================] - 0s 3ms/step
(100, 1)
Accuracy of subcomm3d 32: 33.000001311302185
7/7 [==============================] - 0s 3ms/step
(220, 1)
Accuracy of subcomm3d 64: 28.63636314868927
15/15 [==============================] - 0s 2ms/step
(460, 1)
Accuracy of subcomm3d 128: 24.782608449459076
30/30 [==============================] - 0s 3ms/step
(950, 1)
Accuracy of subcomm3d 256: 19.36842054128647
61/61 [==============================] - 0s 2ms/step
(1950, 1)
Accuracy of subcomm3d 512: 13.076923787593842


In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model(model_dir)

In [ ]:
applications = ['laghos']
ranks = ['32', '64', '128', '256', '512']
#applications = ['swfft']
ranks = ['32','64', '128', '256', '512']
#applications = ['amg', 'kripke', 'laghos', 'stencil4d', 'subcomm3d', 'sw4lite', 'minife']
#ranks = ['32', '64', '128', '256', '512']

In [ ]:
for a in applications:
  for r in ranks:
    #f_name = '/content/gdrive/My Drive/csvstest/full-{}-{}.csv'.format(a, r)
    f_name = '/content/gdrive/My Drive/csvs/full-{}-{}.csv'.format(a, r)
    print(f_name)
    data_a = pd.read_csv(f_name)
    data_a = data_a.sample(frac=1).reset_index(drop=True)
    data_list = []
    range_of_columns = 301
    for k in range(300):
      data_list.append('Data_{}'.format(k))
    data_a['result'] = data_a[data_list].sum(axis=1)
    elephant = []
    for z in data_a['result'].to_list():
        if z > (1 * 10**8):
          elephant.append(1)
        else:
          elephant.append(0)
    data_a['Is_Elephant'] = elephant
    data_a = data_a.fillna(0)
    labels_train = data_a['Is_Elephant'].to_list()
    #print(data_a)
    #print(labels_train)
    for k in range(301):
      if 'Time_{}'.format(k) in data_a.columns:
        data_a = data_a.drop('Time_{}'.format(k), axis='columns')
      if 'Rate_{}'.format(k) in data_a.columns:
        data_a = data_a.drop('Rate_{}'.format(k), axis='columns')
      if 'Packet_Rate_{}'.format(k) in data_a.columns:
        data_a = data_a.drop('Packet_Rate_{}'.format(k), axis='columns')
    result = pd.DataFrame()
    shape_of_df = data_a.iloc[:, 2:8].shape[1] # Current shape of dataframe
    #print(shape_of_df)
    number_of_times = range_of_columns // shape_of_df
    number_of_add = range_of_columns % shape_of_df
    for ii in range(number_of_times):
      result = pd.concat([result, data_a.iloc[:, 2:(shape_of_df+2)]], axis=1)
    result = pd.concat([result, data_a.iloc[:, 2:(number_of_add+2)]], axis=1)
    #print(shape_of_df, number_of_times)
    #print(result.shape)
    #print(result)
    scaler = MinMaxScaler(feature_range=(0,1))
    xp_train = scaler.fit_transform(result)

    labels_1_train = [[i] for i in labels_train]
    labels_1_train = np.array(labels_1_train)
    train_X = xp_train
    train_Y = labels_1_train
    scores = model.evaluate(train_X, train_Y, verbose=0)
    predict= model.predict(train_X, verbose=1)
    #print(predict)
    print("Accuracy of {} {}: {}".format(a, r , scores[1]*100))

/content/gdrive/My Drive/csvs/full-laghos-32.csv
9/9 [==============================] - 0s 2ms/step
Accuracy of laghos 32: 24.372759461402893
/content/gdrive/My Drive/csvs/full-laghos-64.csv
21/21 [==============================] - 0s 2ms/step
Accuracy of laghos 64: 56.711918115615845
/content/gdrive/My Drive/csvs/full-laghos-128.csv
46/46 [==============================] - 0s 1ms/step
Accuracy of laghos 128: 34.83068346977234
/content/gdrive/My Drive/csvs/full-laghos-256.csv
98/98 [==============================] - 0s 1ms/step
Accuracy of laghos 256: 25.07195472717285
/content/gdrive/My Drive/csvs/full-laghos-512.csv
211/211 [==============================] - 0s 1ms/step
Accuracy of laghos 512: 60.05648970603943


In [ ]:
print(train_X.shape)

(51221, 301)


In [ ]:

!saved_model_cli show --dir model --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['dense_1_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 301)
      name: serving_default_dense_1_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [ ]:
from google.colab import files
!zip -r model.zip model
files.download("model.zip")

  adding: model/ (stored 0%)
  adding: model/variables/ (stored 0%)
  adding: model/variables/variables.index (deflated 55%)
  adding: model/variables/variables.data-00000-of-00001 (deflated 32%)
  adding: model/saved_model.pb (deflated 85%)
  adding: model/keras_metadata.pb (deflated 78%)
  adding: model/assets/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("test_num.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(train_X)
np.savetxt('test_num.txt', train_X, fmt='%f')


[[3.67508196e-02 4.63011706e-02 3.80958194e-02 ... 3.26176514e-02
  3.87473501e-02 1.10198917e-01]
 [0.00000000e+00 3.17889014e-05 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.39453110e-05 2.59333857e-03 4.44408841e-04 ... 3.99917377e-04
  4.90472786e-04 1.28737053e-03]
 ...
 [1.50238092e-04 1.62759175e-04 8.13795875e-05 ... 7.34425026e-05
  1.60563981e-04 4.70809793e-04]
 [9.61758537e-03 1.04191303e-02 5.20956515e-03 ... 4.70146770e-03
  1.02786036e-02 3.01391831e-02]
 [9.61758537e-03 1.04191303e-02 5.20956515e-03 ... 4.70146770e-03
  1.02786036e-02 3.01391831e-02]]


In [ ]:
predict= model.predict(train_X, verbose=1)
print(predict)

1601/1601 [==============================] - 2s 1ms/step
[[0.9857718 ]
 [0.00612363]
 [0.0072327 ]
 ...
 [0.00630432]
 [0.04248509]
 [0.04241738]]
